## Imports



In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import mean_absolute_percentage_error

#Facebook Prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

In [ ]:
# https://storage.googleapis.com/energy_consumption_903/df_all_avg_tariff_v3.csv
# https://storage.googleapis.com/energy_consumption_903/df_all_avg_v3.csv

## Data-Split

In [2]:
def split_data(filename):
#     print('Input precise path for data including extension')
#     filename = input()
    fulldata = pd.read_csv(filename)
    fulldata['DateTime'] = pd.to_datetime(fulldata['DateTime'])
    train_data = fulldata[(fulldata['DateTime'] >= '2012-01-01') & (fulldata['DateTime'] < '2014-01-01')].reset_index(drop = True)
    validation_data = fulldata[(fulldata['DateTime'] >= '2014-01-01') & (fulldata['DateTime'] < '2014-02-01')].reset_index(drop = True)
    test_data = fulldata[(fulldata['DateTime'] >= '2014-02-01') & (fulldata['DateTime'] < '2014-03-01')].reset_index(drop = True)
    return train_data, validation_data, test_data

## Data-Processing 

In [54]:
def create_data(Acorn_group,Tariff):
    tdata, vdata, testd=split_data(f'https://storage.googleapis.com/energy_usage_prediction_903/df_{Acorn_group}_avg_{Tariff}_v1.csv')
    # add val and train for prophet
    pdata=pd.concat([tdata,vdata],axis=0).reset_index(drop=True)

    global_average=False

    if global_average==False:
        # not for global average
        pdata.drop(columns='Unnamed: 0',inplace=True)
        testd.drop(columns='Unnamed: 0',inplace=True)
        train_df=pdata.loc[:,['DateTime','KWH/hh']]
        test_df=testd.loc[:,['DateTime','KWH/hh']]
        val_df=vdata.loc[:,['DateTime','KWH/hh']]
    else:
        # group all for dumb models
        df5=pdata.loc[:,['DateTime','Acorn_Group','KWH/hh']]
        df5.set_index('DateTime',inplace=True)
        train_df=df5.groupby(by=df5.index).mean()
        train_df=train_df.reset_index()
        test_df=testd.loc[:,['DateTime','Acorn_Group','KWH/hh']].groupby(by=testd['DateTime']).mean()
        test_df.reset_index(inplace=True)
        test_df=testd.loc[:,['DateTime','Acorn_Group','KWH/hh']].groupby(by=testd['DateTime']).mean()
        test_df.reset_index(inplace=True)
        val_df=vdata.loc[:,['DateTime','Acorn_Group','KWH/hh']].groupby(by=testd['DateTime']).mean()
        val_df.reset_index(inplace=True)
    return train_df,test_df,val_df

In [56]:
train_df,test_df,val_df=create_data('A','ToU')
train_df

,DateTime,KWH/hh
0,2012-01-01 00:00:00,0.279000
1,2012-01-01 00:30:00,0.296000
2,2012-01-01 01:00:00,0.350000
3,2012-01-01 01:30:00,0.286000
4,2012-01-01 02:00:00,0.254000
...,...,...
36571,2014-01-31 21:30:00,0.529333
36572,2014-01-31 22:00:00,0.458708
36573,2014-01-31 22:30:00,0.459583
36574,2014-01-31 23:00:00,0.402917


## Optional Parameters

### Holidays

In [38]:
holidays=pd.read_csv('https://storage.googleapis.com/energy_consumption_903/uk_bank_holidays.csv')
holidays.rename(columns={'Type':'holiday','Bank holidays':'ds'},inplace=True)
holidays.loc[:,'ds']=pd.to_datetime(holidays['ds'],format="%d/%m/%Y")
holidays.head()

,holiday,ds
0,Boxing Day,2012-12-26
1,Christmas Day,2012-12-25
2,Summer bank holiday,2012-08-27
3,Queen's Diamond Jubilee,2012-05-06
4,Spring bank holiday,2012-04-06


### Weather

In [25]:
## if using prophet
twd, vwd, testwd=split_data('https://storage.googleapis.com/weather-data-processed-for-le-wagon/cleaned_weather_hourly_darksky.csv')
wd=pd.concat([twd,vwd],axis=0).reset_index(drop=True)
wd_filt=wd[['DateTime','temperature','windSpeed','precipType_rain']].dropna()
wd_filt['DateTime']=pd.to_datetime(wd_filt['DateTime'])
test_wd=testwd[['DateTime','temperature','windSpeed','precipType_rain']].dropna()
test_wd['DateTime']=pd.to_datetime(test_wd['DateTime'])
# # wind = wd_filt['windSpeed'].interpolate(method='linear')
# # rain = wd_filt['precipType_rain'].interpolate(method='linear')
train_wd=train_df[['DateTime']].merge(wd_filt,on='DateTime',how='inner')
test_wd=test_df[['DateTime']].merge(test_wd,on='DateTime',how='inner')

,DateTime,temperature,windSpeed,precipType_rain
0,2012-01-01 00:00:00,9.75,1.79,1.0
1,2012-01-01 00:30:00,9.75,1.79,1.0
2,2012-01-01 01:00:00,9.68,1.84,1.0
3,2012-01-01 01:30:00,9.68,1.84,1.0
4,2012-01-01 02:00:00,9.63,1.46,1.0
...,...,...,...,...
36571,2014-01-31 21:30:00,8.64,5.39,1.0
36572,2014-01-31 22:00:00,3.17,6.12,1.0
36573,2014-01-31 22:30:00,3.17,6.12,1.0
36574,2014-01-31 23:00:00,3.14,5.61,1.0


## Train

In [59]:
def train_model(train_df,train_wd,holidays,add_weather=True): 
    train_df.rename(columns={"DateTime": "ds", "KWH/hh": "y"},inplace=True)
    if add_weather==True:  
        temp = train_wd['temperature'].interpolate(method='linear')
        train_df['temp']=temp
        m = Prophet(holidays=holidays,changepoint_prior_scale=0.01).add_regressor('temp', prior_scale=0.5, mode='multiplicative')
        m.fit(train_df)
    else:
        m = Prophet(holidays=holidays,changepoint_prior_scale=0.01)
        m.fit(train_df)
    return m

In [61]:
m=train_model(train_df,train_wd,holidays,add_weather=False)


13:14:20 - cmdstanpy - INFO - Chain [1] start processing
13:14:51 - cmdstanpy - INFO - Chain [1] done processing


## Predict

In [71]:
def forecast_model(m,train_wd,test_wd,add_weather=True): 
    future = m.make_future_dataframe(periods=48*27+1, freq='30T')
    if add_weather==True:   
        wd_filt_future=future[['ds']].merge(pd.concat([future,pd.concat([train_wd,test_wd],axis=0)]),left_on='ds',right_on='DateTime',how='inner').drop(columns='DateTime')
        temp_future=wd_filt_future['temperature'].interpolate(method='linear')
        future['temp']=temp_future
        fcst = m.predict(future)
    else:
        fcst = m.predict(future)
    return fcst

In [ ]:
f=forecast_model(m,train_wd,test_wd,add_weather=False)

## Plot

In [ ]:
def plot_graphs(f):
    forecast=f.loc[f['ds']>='2014-02-01 00:00:00',['ds','yhat']]
    figure(figsize=(15,6))
    sns.lineplot(x=forecast['ds'],y=forecast['yhat'],label='Forecast');
    sns.lineplot(x=test_df['DateTime'],y=test_df['KWH/hh'],label='Actual');
    figure(figsize=(15,6))
    sns.lineplot(x=test_wd['DateTime'],y=test_wd['temperature'],label='Weather');
    return forecast

In [ ]:
forecast=plot_graphs(f)

In [ ]:
def evaluate(actual,forecasted):
    return np.round(mean_absolute_percentage_error(actual,forecasted),4)
evaluate(test_df['KWH/hh'],forecast['yhat'])